Explorar a base de dados utilizando recursos do PySpark.

In [1]:
import os
import sys

os.environ['SPARK_HOME'] = 'C:\spark'
os.environ['HADOOP_HOME'] = 'C:\spark\hadoop'
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("InghtPlaces_Anuncios") \
    .getOrCreate()
    
dados = spark.read.json("dados/dataset_bruto.json")


In [3]:
print(f'Linhas: {dados.count()} | Colunas: {len(dados.columns)}')
print('-'*50)
# print(dados.printSchema())

Linhas: 89083 | Colunas: 3
--------------------------------------------------


In [4]:
# Para nossa análise, apenas as informações do campo "anuncio" serão relevantes. 
# Por isso, vamos focar em analisar as colunas desse campo.

dados = dados.select('anuncio.*')
# dados.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

In [5]:
# O time de Data Science solicitou que fizéssemos alguns filtros nas colunas 
# tipo_uso, tipo_unidade e tipo_anuncio da nossa base de dados:

# tipo_uso: Residencial;
# tipo_unidade: Apartamento;
# tipo_anuncio: Usado.

# Para entender o impacto que esses filtros terão sob a nossa base de dados, 
# crie uma tabela de frequências para cada uma dessas colunas antes de filtrá-las.

dados.groupBy(['tipo_uso','tipo_unidade','tipo_anuncio']).count().show()


+-----------+------------+------------+-----+
|   tipo_uso|tipo_unidade|tipo_anuncio|count|
+-----------+------------+------------+-----+
|Residencial|      Outros|       Usado| 7502|
|Residencial| Apartamento|  Lançamento|  235|
|Residencial| Apartamento|       Usado|66562|
|  Comercial| Apartamento|       Usado|    4|
|  Comercial|      Outros|  Lançamento|    3|
|  Comercial|      Outros|       Usado| 4443|
|  Comercial|        Casa|       Usado|   92|
|Residencial|        Casa|  Lançamento|    3|
|Residencial|      Outros|  Lançamento|   15|
|Residencial|        Casa|       Usado|10224|
+-----------+------------+------------+-----+



In [7]:
# 66562 a variavel alvo solicitada pelo time de Data Science.
(dados.count() / 89083) * 100

74.7190822042365

Realizando filtros para selecionar apenas os dados que serão utilizados.

In [8]:
# 74.8% dos dados serão utilizados para a análise.

dados = dados.filter(dados.tipo_uso == 'Residencial') \
    .filter(dados.tipo_unidade == 'Apartamento') \
    .filter(dados.tipo_anuncio == 'Usado')
dados.count()

66562

Remoção de Arrays as transformando em tipo Inteiro.

In [9]:
#Nesse contexto, vamos transformar os dados das colunas:

# "quartos",     |-- quartos: array (nullable = true)
#                | | -- element: long(containsNull=true)
# "suites",      |-- suites: array (nullable = true)
#                | | -- element: long(containsNull=true)
# "banheiros",   |-- banheiros: array (nullable = true)
#                | | -- element: long(containsNull=true)
# "vaga",        |-- vaga: array (nullable = true)
#                | | -- element: long(containsNull=true)
# "area_total",  |-- area_total: array (nullable = true)
#                | | -- element: string(containsNull=true)
# "area_util",   |-- area_util: array (nullable = true)
#                | | -- element: string(containsNull=true)

from pyspark.sql.types import IntegerType
from pyspark.sql import functions as f

dicionario = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

for item in dicionario:
    dados = dados.withColumn(item, dados[item][0].cast("int"))

Tratamento de informações sobre localização.

In [10]:
# A coluna de endereço possui as informações de bairro e zona que devem ser extraídas para o dataset final.

dicionario = ['bairro', 'zona']

for item in dicionario:
    dados = dados.withColumn(item, dados['endereco'][item])


Transformar cada campo da coluna "valores" em uma coluna separada.

In [11]:
# Nesse contexto, transforme os dados da coluna valores em colunas separadas.

# |-- valores: array(nullable=true)
# | | -- element: struct(containsNull=true)
# | | |-- condominio: string(nullable=true)
# | | |-- iptu: string(nullable=true)
# | | |-- tipo: string(nullable=true)
# | | |-- valor: string(nullable=true)

dicionario = ['condominio', 'iptu', 'tipo', 'valor']

for item in dicionario:
    dados = dados.withColumn(item, dados['valores'][0][item])

In [12]:
#Como se trata de um estudo sobre o preço de venda dos imóveis, o time de cientistas de dados solicitou apenas as informações do tipo VENDA.
dados = dados.filter(dados.tipo == 'Venda')


In [13]:
#Remover colunas desnecessárias : Endereço, Valores e Tipo.
dados = dados.drop('endereco', 'valores',
                   'tipo', 'tipo_anuncio', 'tipo_unidade', 'tipo_uso', 'area_total')


Inicio Semana 2 - Tratamento de dados para o modelo ML.

In [14]:
# Converter os tipos das colunas

#Podemos converter o tipo de colunas numéricas, como "andar", "banheiros", "suites" e "quartos" para o tipo inteiro. 
# Além disso também pode ser necessário converter as colunas "area_util", "condominio", "iptu" e "valor" para o tipo double.

inteiros = ['andar', 'banheiros', 'suites', 'quartos']
doubles = ['area_util', 'condominio', 'iptu', 'valor']

for item in inteiros:
    dados = dados.withColumn(item, dados[item].cast('int'))

for item in doubles:
    dados = dados.withColumn(item, dados[item].cast('double'))


In [15]:
#coletar apenas o texto dentro da coluna caracteristicas
lista = dados.select(f.explode("caracteristicas")).distinct().collect()
lista = [item[0] for item in lista]
print(lista)

['Condomínio fechado', 'Playground', 'Portão eletrônico', 'Piscina', 'Animais permitidos', 'Portaria 24h', 'Elevador', 'Academia', 'Salão de festas', 'Churrasqueira']


In [16]:
# se a coluna caracteristicas do dataframe contiver um item da lista, criar uma nova coluna com valor 1 e 0 caso contrário.
import unidecode
from unidecode import unidecode
from pyspark.sql.functions import array_contains

for item in lista:
  dados = dados.withColumn(unidecode(item.replace(' ', '_').lower()), array_contains(
      'caracteristicas', item).cast('integer'))


In [17]:
dados = dados.drop('caracteristicas')

In [18]:
dados = dados\
    .where(f.col('zona') != '')

In [19]:
lista_2 = dados.select('zona').distinct().collect()
lista_2 = [item[0] for item in lista_2]
print(lista_2)


['Zona Norte', 'Zona Oeste', 'Zona Central', 'Zona Sul']


In [20]:
for i in lista_2:
    dados = dados.withColumn(i, f.when(f.col('zona') == i, 1).otherwise(0))

In [21]:
dados = dados.drop('zona','bairro')

Tratando dados Nulos

In [22]:

dados.select([f.count(f.when(dados[c].isNull(), True)).alias(c) for c in dados.columns]).toPandas().T


,0
andar,0
area_util,0
banheiros,0
id,0
quartos,0
suites,5541
vaga,3004
condominio,2347
iptu,7152
valor,0


In [23]:
dados = dados\
    .select('*')\
    .na\
    .fill(0)


In [25]:
dados.write.parquet('DADOS/dataset_preparado',mode='overwrite')